In [ ]:
# def cancel_flight(PNR_Number,Booking_Status,Refund_Status):
#     #cancel_flight_to_db(PNR_Number,Booking_Status,Refund_Status)
#     print("The tool is called here....")
#     return {"cancelled":"OK"}


# cancel_flight_json = {
#     "name": "cancel_flight",
#     "description": "Use this tool to cancel a flight for a user who has provided required parameters",
#     "parameters": {
#         "type": "object",
#         "properties": {
#             "PNR_Number": {
#                 "type": "string",
#                 "description": "This is the PNR Number of this user or traveller"
#             },
#             "Booking_Status": {
#                 "type": "string",
#                 "description": "This is the Booking Status of this user or traveller"
#             },
#             "Refund_Status": {
#                 "type": "string",
#                 "description":  "This is the Refund Status of this user or traveller"
#             }

#         },
#         "required": ["PNR_Number","Booking_Status","Refund_Status"],
#         "additionalProperties": False
#     }
# }

# tools = [{"type": "function", "function": cancel_flight_json}]


In [ ]:
# Build prompt
template = """You are a SQLite expert. Given an input request, return a syntactically correct SQLite query to run.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Do not return any new columns nor perform aggregation on columns. 

In case of cancellation query , do confirm with user that the PNR_Number is correct and share information about cancellation policy and refund amount. 
In case of cancellation query, update the table Flight_reservation the Booking_Status to "Cancelled" and Refund_Status to "Refunded".

The database {airline.db} currently have data of Flight Reservation, and the PNR_Number column includes the ticker representing the Flight Reservation.
Once you start generating sql queries make sure that you only use correct PNR_Number for filtering and restrict yourself to use only current Flight Reservation.
If user asks for unique PNR_Number present in table then do not limit the output.t

Use the following format:

Request: Request here
SQLQuery: Generated SQL Query here

Only use the following tables:
create table IF NOT EXISTS Flight_reservation (
        PNR_Number VARCHAR(10),
        Customer_Name VARCHAR(50),
        Flight_ID VARCHAR(50),
        Airline VARCHAR(50),
        From_City VARCHAR(50),
        To_City VARCHAR(50),
        Departure_Time TIME,
        Arrival_Time TIME,
        Travel_Date DATE,
        Booking_Date DATE,
        Booking_Status VARCHAR(20),
        Refund_Status VARCHAR(20)
    )''')

Request: {request}
SQLQuery:
"""

PROMPT = PromptTemplate(input_variables=["request"], template=template)

In [ ]:
# SQL Query Generation Chain - created using LCEL (LangChain Expression Language)

sql_chain = (PROMPT
             | llm
             | StrOutputParser()       # to get output in a more usable format
             )